In [248]:
import pandas as pd
import logging

In [249]:
max_buses = 600
file_name = 'collected_schedules/best_schedule_bus600_gen15.csv'
schedule_data = pd.read_csv(file_name)
schedule_data.head()    

,bus,trip_id
0,0,14811
1,0,7146
2,0,466
3,0,485
4,0,11534


In [250]:
trips = pd.read_csv('formatted_DMRC_trips.csv')
stops = pd.read_csv('DMRC_Data/stops.txt')
times = pd.read_csv('DMRC_Data/stop_times.txt')

In [251]:
trips.head()

,Unnamed: 0,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,start_time_y,end_time_y,start_stop_y,end_stop_y
0,0,0,weekday,0,NaN,NaN,NaN,NaN,shp_1_30,0,0,05:28:08,06:18:38,21,1
1,1,0,weekday,1,NaN,NaN,NaN,NaN,shp_1_30,0,0,05:35:18,06:25:48,21,1
2,2,0,weekday,10,NaN,NaN,NaN,NaN,shp_1_30,0,0,06:39:48,07:30:18,21,1
3,3,0,weekday,100,NaN,NaN,NaN,NaN,shp_1_30,0,0,16:48:04,17:38:34,21,1
4,4,2,weekday,1000,NaN,NaN,NaN,NaN,shp_1_13,0,0,18:26:21,20:00:41,71,36


In [252]:
stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon
0,1,NaN,Dilshad Garden,NaN,28.675991,77.321495
1,2,NaN,Jhilmil,NaN,28.675648,77.312393
2,3,NaN,Mansrover park,NaN,28.675352,77.301178
3,4,NaN,Shahdara,NaN,28.673531,77.287270
4,5,NaN,Welcome,NaN,28.671986,77.277931


In [253]:
times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,continuous_pickup,continuous_drop_off
0,0,05:28:08,05:28:28,21,0,NaN,0,0,0.000,1,NaN,NaN
1,0,05:30:58,05:31:18,20,1,NaN,0,0,1202.405,1,NaN,NaN
2,0,05:33:28,05:33:48,19,2,NaN,0,0,2480.750,1,NaN,NaN
3,0,05:35:33,05:35:53,18,3,NaN,0,0,3314.936,1,NaN,NaN
4,0,05:37:53,05:38:13,17,4,NaN,0,0,4300.216,1,NaN,NaN


In [254]:
schedule = pd.merge(schedule_data, trips, on='trip_id')
schedule.head()

,bus,trip_id,Unnamed: 0,route_id,service_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,start_time_y,end_time_y,start_stop_y,end_stop_y
0,0,14811,1712,29,weekday,NaN,NaN,NaN,NaN,shp_1_35,0,0,06:06:15,07:58:11,218,173
1,0,7146,4765,15,weekday,NaN,NaN,NaN,NaN,shp_1_29,0,0,08:06:21,09:14:21,172,153
2,0,466,3811,1,weekday,NaN,NaN,NaN,NaN,shp_1_32,0,0,09:29:30,10:39:05,21,225
3,0,485,3919,1,weekday,NaN,NaN,NaN,NaN,shp_1_32,0,0,11:27:18,12:36:53,21,225
4,0,11534,656,22,weekday,NaN,NaN,NaN,NaN,shp_1_22,0,0,13:39:20,14:04:19,62,71


In [255]:
# Fix time format for arrival and departure times
def fix_time_format(time_str):
    try:
        hours, minutes, seconds = map(int, time_str.split(":"))
        if hours >= 24:
            hours -= 24
        return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    except ValueError:
        # Return the original string if it's not a valid time format
        logging.warning(f"Invalid time format encountered: {time_str}")
        return time_str

In [256]:
schedule = schedule.drop(columns=['Unnamed: 0', 'trip_headsign', 'trip_short_name',
                         'direction_id', 'block_id', 'wheelchair_accessible',	'bikes_allowed'])
schedule = schedule.rename(
    columns={'start_time_y': 'start_time', 'end_time_y': 'end_time', 'start_stop_y': 'start_stop', 'end_stop_y': 'end_stop'})

schedule = schedule.merge(stops.drop(columns=['stop_code', 'stop_desc']),
                                            left_on='start_stop',
                                            right_on='stop_id')
schedule = schedule.rename(
    columns={
        'stop_name': 'start_stop_name',
        'stop_lat': 'start_stop_lat',
        'stop_lon': 'start_stop_lon'
    }
).drop(columns='stop_id')

schedule = schedule.merge(stops.drop(columns=['stop_code', 'stop_desc']),
                                            left_on='end_stop',
                                            right_on='stop_id')
schedule = schedule.rename(
    columns={
        'stop_name': 'end_stop_name',
        'stop_lat': 'end_stop_lat',
        'stop_lon': 'end_stop_lon'
    }
).drop(columns='stop_id')
schedule['start_time'] = schedule['start_time'].apply(fix_time_format)
schedule['end_time'] = schedule['end_time'].apply(fix_time_format)
schedule.head()

,bus,trip_id,route_id,service_id,shape_id,start_time,end_time,start_stop,end_stop,start_stop_name,start_stop_lat,start_stop_lon,end_stop_name,end_stop_lat,end_stop_lon
0,0,14811,29,weekday,shp_1_35,06:06:15,07:58:11,218,173,Shiv Vihar,28.721863,77.289635,Majlis Park,28.724157,77.182068
1,0,7146,15,weekday,shp_1_29,08:06:21,09:14:21,172,153,Sector 55-56 (Rapid Metro),28.424587,77.105042,Phase 3 (Rapid Metro),28.494329,77.093552
2,0,466,1,weekday,shp_1_32,09:29:30,10:39:05,21,225,Rithala,28.720821,77.105042,Shaheed Sthal (New Bus Adda),28.670177,77.416031
3,0,485,1,weekday,shp_1_32,11:27:18,12:36:53,21,225,Rithala,28.720821,77.105042,Shaheed Sthal (New Bus Adda),28.670177,77.416031
4,0,11534,22,weekday,shp_1_22,13:39:20,14:04:19,62,71,Qutab Minar,28.512714,77.185791,Huda City Centre,28.459118,77.072586


In [257]:
# Compute distance between two stops based on lat/lon
def haversine(lat1, lon1, lat2, lon2):
    from math import radians, cos, sin, sqrt, atan2

    R = 6371  # Earth's radius in km

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

# Calculate travel time between stops
def travel_time(stop1, stop2, stops):
    lat1, lon1 = stops.loc[stops['stop_id'] == stop1, ['stop_lat', 'stop_lon']].values[0]
    lat2, lon2 = stops.loc[stops['stop_id'] == stop2, ['stop_lat', 'stop_lon']].values[0]
    distance = haversine(lat1, lon1, lat2, lon2)
    return distance / 50 * 60  # Assume average speed of 50 km/h

In [258]:
# Initialize variables
bus_utilization = 0
bus_unutilization = 0
overlap_penalty = 0
total_active_time = 0
total_idle_time = 0
total_active_distance = 0

# To store overlapping schedule details
overlapping_schedules = []

# Group schedule by buses
buses = schedule.groupby('bus')

# Iterate through each bus's schedule
for bus_id, bus_schedule in buses:
    bus_schedule = bus_schedule.sort_values(by='start_time')  # Sort by start_time
    if bus_schedule.empty:  # Check if the bus has no trips
        continue
    bus_utilization += 1

    # Iterate through trips for this bus
    for i in range(len(bus_schedule)):
        current_trip = bus_schedule.iloc[i]

        # Extract trip details
        start_stop = current_trip['start_stop']
        end_stop = current_trip['end_stop']
        trip_start_time = pd.to_datetime(current_trip['start_time'])
        trip_end_time = pd.to_datetime(current_trip['end_time'])

        # Calculate active time for the trip
        trip_duration = (trip_end_time - trip_start_time).total_seconds() / 60
        total_active_time += trip_duration

        # Calculate distance traveled during the trip
        lat1, lon1 = current_trip['start_stop_lat'], current_trip['start_stop_lon']
        lat2, lon2 = current_trip['end_stop_lat'], current_trip['end_stop_lon']
        total_active_distance += haversine(lat1, lon1, lat2, lon2)

        # Handle the idle time and overlap penalty for subsequent trips
        if i < len(bus_schedule) - 1:
            next_trip = bus_schedule.iloc[i + 1]
            next_trip_start_time = pd.to_datetime(next_trip['start_time'])
            next_trip_end_time = pd.to_datetime(next_trip['end_time'])

            # Calculate idle time
            idle_time = (next_trip_start_time - trip_end_time).total_seconds() / 60
            total_idle_time += max(idle_time, 0)  # Only count positive idle times

            # Calculate overlap penalty and capture details of overlapping trips
            if trip_end_time > next_trip_start_time:
                overlap_penalty += 1
                overlapping_schedules.append({
                    'bus_id': bus_id,
                    'current_trip_id': current_trip['trip_id'],
                    'current_start_time': trip_start_time,
                    'current_route_id': current_trip['route_id'],
                    'current_service_id': current_trip['service_id'],
                    'current_shape_id': current_trip['shape_id'],
                    'current_end_time': trip_end_time,
                    'current_start_stop': current_trip['start_stop'],
                    'current_end_stop': current_trip['end_stop'],
                    'current_start_stop_name': current_trip['start_stop_name'],
                    'current_start_stop_lat': current_trip['start_stop_lat'],
                    'current_start_stop_lon': current_trip['start_stop_lon'],
                    'current_end_stop_name': current_trip['end_stop_name'],
                    'current_end_stop_lat': current_trip['end_stop_lat'],
                    'current_end_stop_lon': current_trip['end_stop_lon'],
                    'next_trip_id': next_trip['trip_id'],
                    'next_start_time': next_trip_start_time,
                    'next_route_id': next_trip['route_id'],
                    'next_service_id': next_trip['service_id'],
                    'next_shape_id': next_trip['shape_id'],
                    'next_end_time': next_trip_end_time,
                    'next_start_stop': next_trip['start_stop'],
                    'next_end_stop': next_trip['end_stop'],
                    'next_start_stop_name': next_trip['start_stop_name'],
                    'next_start_stop_lat': next_trip['start_stop_lat'],
                    'next_start_stop_lon': next_trip['start_stop_lon'],
                    'next_end_stop_name': next_trip['end_stop_name'],
                    'next_end_stop_lat': next_trip['end_stop_lat'],
                    'next_end_stop_lon': next_trip['end_stop_lon']
                })

# Normalize parameters for fitness calculation
bus_utilization_normalized = bus_utilization / len(buses)  # Proportion of buses used
overlap_penalty_normalized = overlap_penalty / len(schedule) * 100
activity_ratio = total_active_time / max(total_idle_time, 1)  # Avoid division by zero
total_active_distance_normalized = total_active_distance / 1000  # Scale distance to kilometers

# Combine into a single fitness value (smaller values are better)
fitness_value = (
    1 / max(bus_utilization_normalized, 1) +
    overlap_penalty_normalized * 1000 +
    1 / max(activity_ratio, 1) +  # Inverse for minimization
    total_active_distance_normalized
)

print(
    f"Fitness: {fitness_value:.6f}, Bus Utilization: {bus_utilization} | Overlaps: {overlap_penalty:.6f}, "
    f"Activity Ratio: {activity_ratio}, Active Distance: {total_active_distance:.2f} km"
)

# Convert the overlapping schedule list into a DataFrame
overlapping_schedule_df = pd.DataFrame(overlapping_schedules)

# Output overlapping schedule for review
print("Overlapping Schedule Details:")
print(overlapping_schedule_df)

Fitness: 206.392926, Bus Utilization: 598 | Overlaps: 5.000000, Activity Ratio: 0.5048555478593422, Active Distance: 111920.21 km
Overlapping Schedule Details:
   bus_id  current_trip_id  current_start_time  current_route_id  \
0     122             6201 2025-01-23 22:38:55                12   
1     163             1644 2025-01-23 22:10:14                 3   
2     212            10962 2025-01-23 22:58:56                21   
3     361            11660 2025-01-23 23:03:24                22   
4     509             1652 2025-01-23 22:43:42                 3   

  current_service_id current_shape_id    current_end_time  current_start_stop  \
0            weekday         shp_1_16 2025-01-23 23:47:07                 108   
1            weekday         shp_1_25 2025-01-23 22:59:21                  62   
2            weekday         shp_1_34 2025-01-23 23:48:03                  44   
3            weekday         shp_1_22 2025-01-23 23:28:23                  62   
4            weekday      

In [259]:
overlapping_schedule_df.head()

,bus_id,current_trip_id,current_start_time,current_route_id,current_service_id,current_shape_id,current_end_time,current_start_stop,current_end_stop,current_start_stop_name,...,next_shape_id,next_end_time,next_start_stop,next_end_stop,next_start_stop_name,next_start_stop_lat,next_start_stop_lon,next_end_stop_name,next_end_stop_lat,next_end_stop_lon
0,122,6201,2025-01-23 22:38:55,12,weekday,shp_1_16,2025-01-23 23:47:07,108,81,Janak Puri West,...,shp_1_16,2025-01-23 00:14:07,108,81,Janak Puri West,28.629637,77.077866,Botanical Garden,28.564198,77.334656
1,163,1644,2025-01-23 22:10:14,3,weekday,shp_1_25,2025-01-23 22:59:21,62,44,Qutab Minar,...,shp_1_16,2025-01-23 00:07:22,108,81,Janak Puri West,28.629637,77.077866,Botanical Garden,28.564198,77.334656
2,212,10962,2025-01-23 22:58:56,21,weekday,shp_1_34,2025-01-23 23:48:03,44,62,Vishwavidyalaya,...,shp_1_34,2025-01-23 00:00:36,44,62,Vishwavidyalaya,28.694765,77.212418,Qutab Minar,28.512714,77.185791
3,361,11660,2025-01-23 23:03:24,22,weekday,shp_1_22,2025-01-23 23:28:23,62,71,Qutab Minar,...,shp_1_25,2025-01-23 00:02:06,62,44,Qutab Minar,28.512714,77.185791,Vishwavidyalaya,28.694765,77.212418
4,509,1652,2025-01-23 22:43:42,3,weekday,shp_1_25,2025-01-23 23:32:49,62,44,Qutab Minar,...,shp_1_26,2025-01-23 00:41:35,173,218,Majlis Park,28.724157,77.182068,Shiv Vihar,28.721863,77.289635


In [260]:
def get_unallocated_buses(all_buses, schedule):
    # Extract buses with allocated trips from the schedule
    allocated_buses = schedule['bus'].dropna().unique()
    
    # Identify unallocated buses by comparing with all_buses
    unallocated_buses = list(set(all_buses) - set(allocated_buses))
    
    return unallocated_buses

In [261]:
# Convert time columns to datetime
time_columns = ['current_start_time', 'current_end_time',
                'next_start_time', 'next_end_time']
for col in time_columns:
    overlapping_schedule_df[col] = pd.to_datetime(overlapping_schedule_df[col])


def reallocate_overlapping_trips(overlapping_schedule, available_buses, stop_data):
    # Initialize a list for new allocations
    allocations = []

    for index, row in overlapping_schedule.iterrows():
        if not available_buses:
            print("No available buses left for reallocation.")
            break

        # Get the next available bus
        new_bus_id = available_buses.pop(0)

        # Lookup stop details for the trip
        trip_id = row["next_trip_id"]
        trip_info = trips.get(trip_id, {})

        # Append allocation details
        allocations.append({
            "bus": new_bus_id,
            "trip_id": trip_id,
            "route_id": row["next_route_id"],
            "service_id": row["next_service_id"],
            "shape_id": row["next_shape_id"],
            "start_time": row["next_start_time"],
            "end_time": row["next_end_time"],
            "start_stop": row["next_start_stop"],
            "end_stop": row["next_end_stop"],
            "start_stop_name": row["next_start_stop_name"],
            "start_stop_lat": row["next_start_stop_lat"],
            "start_stop_lon": row["next_start_stop_lon"],
            "end_stop_name": row["next_end_stop_name"],
            "end_stop_lat": row["next_end_stop_lat"],
            "end_stop_lon": row["next_end_stop_lon"],
        })

    # Convert allocations to a DataFrame
    new_allocations_df = pd.DataFrame(allocations)
    return new_allocations_df


all_buses = [i + 1 for i in range(max_buses + 100)]
available_buses = get_unallocated_buses(all_buses, schedule)
print(available_buses)

# Allocate overlapping trips and append to the schedule
new_allocations_df = reallocate_overlapping_trips(
    overlapping_schedule_df, available_buses, stops)
schedule = pd.concat([schedule, new_allocations_df], ignore_index=True)

# Output the updated schedule
print("Updated Schedule:")
print(schedule)

[600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 358, 390]
Updated Schedule:
      bus  trip_id  route_id service_id  shape_id           start_time  \
0       0    14811        29    weekday  shp_1_35             06:06:15   
1       0     7146        15    weekday  shp_1_29             08:06:21   
2       0      466         1    weekday  shp_1_32             09:29:30   
3       0      485         1    weekday  shp_1_32             11:27:18   
4       0    11534        22    weekday  shp_1_22             13:39:20   
...   ...      ...    

In [262]:
schedule.to_csv(file_name.replace('best','fixed'))